In [ ]:
# Copyright 2025 50Hertz Transmission GmbH and Elia Transmission Belgium
#
# This Source Code Form is subject to the terms of the Mozilla Public License, v. 2.0.
# If a copy of the MPL was not distributed with this file,
# you can obtain one at https://mozilla.org/MPL/2.0/.
# Mozilla Public License, version 2.0

# Example 2: Topology optimization of a small grid

The pipeline consists of three main stages:

1. **Data Importing and Preprocessing**  
    This stage involves loading your grid data and preparing it for optimization. Preprocessing may include cleaning the data, handling unsupported elements, and performing calculations such as busbar outage analysis.

2. **DC Optimization**  
    In this stage, the pipeline runs optimization (Map Elites) algorithm using GPU accelerated DC Load flow solver on the preprocessed grid data. The optimizer uses specified metrics and parameters to search for optimal grid topologies or configurations.

3. **AC Validation**  
    After optimization, the best solutions are validated using AC (Alternating Current) power flow analysis to ensure feasibility and performance under more realistic conditions.

---

Before running the pipeline on your data, you need to configure the following:

1. **PipelineConfig**  
    Define the experiment name (`iteration_name`) and the grid file name (`file_name`). The experiment name determines the folder where results and intermediate files are stored. The grid file should be placed inside this folder and can be in formats such as `.xiidm`, `.json` (for PandaPower), or `.zip` (for CGMES).

    Note that a folder named 'iteration_name' should be created inside the 'data' folder and the grid file named 'file_name' should be put inside the 'iteration_name' folder.

2. **Importer Config**  
    Set parameters for importing the grid data, such as area settings and other options relevant to your data source. This configuration ensures that the importer correctly interprets and processes your grid file.

3. **Preprocessing Parameters**  
    Specify options for the preprocessing step, which are independent of the data source. For example, you can enable or disable preprocessing for busbar outage calculations or set limits for action sets.

4. **DC Optimization Configuration (`dc_optimization_cfg`)**  
    Define the parameters for the optimizer and loadflow solver. This includes settings such as which metrics to use in the objective function, runtime limits, the number of worst contingencies to consider, and other optimizer-specific options.

---

**Workflow Summary:**

- Set up the required configurations as described above.
- Run the pipeline to import, preprocess, and optimize your grid data.
- Optionally, rerun the optimization with different parameters without repeating preprocessing or optimisation.
- Validate the optimized solutions using AC power flow analysis.

This modular approach allows you to efficiently experiment with different configurations and optimization strategies while minimizing redundant computations.

In [ ]:
from toop_engine_topology_optimizer.benchmark.benchmark_utils import *

> **Reminder:**  
> Create a folder named `iteration_name` inside the `data` folder, and place the grid file named `file_name` inside the `iteration_name` folder.  
>  
> For example:  
> ```
> data/
> └── <iteration_name>/
>     └── <file_name>
> ```

The data folder already exists in the repository

In [ ]:
# Set up pipeline configuration. Here iteration_name corresponds to the name of the experiment. For example, "test_tso". This folder has to be created inside the "data" folder. This folder is supposed to contain the grid file specified in file_name.
# file_name corresponds to the name of the grid file. This file has to be present inside the folder specified by iteration_name. This folder can have grid data from multiple timesteps.
# The grid file can be in any of the supported formats: .xiidm, .json, .mat, .zip (in case of cgmes)

iteration_name = ""
file_name = "grid.xiidm"

pipeline_cfg = PipelineConfig(
    root_path=Path("../data/grid_node_breaker"),
    iteration_name = iteration_name,
    file_name = file_name,
    grid_type="powsybl")

# Set up dc_optimisation_configs
iteration_path, file_path, data_folder, optimizer_snapshot_dir = get_paths(pipeline_cfg)
static_information_file = data_folder / pipeline_cfg.static_info_relpath

# Example configuration for the optimizer
# Note that this is a minimal example; adjust as needed.
# Look at topology_optimizer.interfaces.messages.dc_params.BatchedMEParameters and topology_optimizer.interfaces.messages.dc_params.LoadflowSolverParameters for options.
# The values for these configuration classes are set in the "lf_config" and "ga_config" sections below.
dc_optimization_cfg = DictConfig({
    "task_name": "test",
    "fixed_files": [str(static_information_file)],
    "double_precision": None,
    "tensorboard_dir": str(iteration_path) + "/results/{task_name}",
    "stats_dir": str(iteration_path) + "/results/{task_name}",
    "summary_frequency": None,
    "checkpoint_frequency": None,
    "stdout": None,
    "double_limits": None,
    "num_cuda_devices": 1,
    "omp_num_threads": 1,
    "xla_force_host_platform_device_count": None,
    "output_json": str(iteration_path) + "/results/output.json",
    "lf_config": {"distributed": False},
    "ga_config": {
        "runtime_seconds": 30,
        "me_descriptors": [{"metric": "split_subs", "num_cells": 10}],
        "observed_metrics": ["overload_energy_n_1", "split_subs"],
        "n_worst_contingencies": 2
    },
})
ac_validation_cfg = DictConfig({
    "n_processes": 1,
    "k_best_topos": 5
})

logger.info("Preparing importer parameters...")
importer_parameters = prepare_importer_parameters(
    file_path,
    data_folder)
# Set cutoff_voltage to an appropriate level to include all voltage levels in the optimization
importer_parameters.area_settings.cutoff_voltage = 10 # in kV

logger.info("Setting up preprocessing parameters...")
preprocessing_parameters = PreprocessParameters(
    action_set_clip=2**10,
    enable_bb_outage=True,
    bb_outage_as_nminus1=False)

In [ ]:
import time

start = time.time()
try:
    topo_path = run_pipeline(pipeline_cfg=pipeline_cfg, dc_optim_config=dc_optimization_cfg, ac_validation_cfg=ac_validation_cfg, importer_parameters=importer_parameters, preprocessing_parameters=preprocessing_parameters)
    logger.info(f"Pipeline finished in {time.time() - start:.2f} seconds")
except Exception as exc:
    logger.exception(f"Pipeline failed: {exc}")
    raise

The results generated by the pipeline are organized in a hierarchical folder structure within the experiment directory (e.g., `data/rte/20250220T0830`). This structure is designed to facilitate easy access to all intermediate and final outputs of the optimization and validation process.

---

### **Folder Structure Overview**

```
data/
└── <iteration_name>/                # e.g., rte
    |── <file_name_without_ext>/     # e.g., 20250212T0830
    |    ├── optimizer_snapshot/
    |    │   ├── run_1/
    |    │   │   ├── res.json
    |    │   │   ├── topology_0/
    |    │   │   │   ├── modified_network.xiidm
    |    │   │   │   ├── ac_loadflow_results.csv
    |    │   │   │   ├── dc_loadflow_results.csv
    |    │   │   │   ├── sld
    |    │   │   │   │   ├── sld_split_station_1.png
    |    │   │   │   │   ├── sld_split_station_2.png
    |    │   │   │   │   └── ...
    |    │   │   ├── topology_1/
    |    │   │   └── ...
    |    │   └── ...
    |    ├── run_2/
    |    │   └── ...
    |    └── ...
    |    └── ...
    └── results/
        └── task_name/
            └── ...
```
#### **optimizer_snapshot/**
- This folder contains the results of all optimization runs for the given experiment.
- Each optimization run is stored in a separate subfolder (e.g., `run_1`, `run_2`, etc.). The names of these folders correspond to different executions of the optimizer, which may vary based on parameters or random seeds.

#### **optimizer_snapshot/run_{n}/**
- Each `run_{n}` folder corresponds to a single execution of the optimizer with a specific configuration.
- **res.json**:  
  - This file summarizes the results of the optimization run, including metrics, best topologies found, and references to the evaluated topologies.
- **topology_{x}/**:  
  - Each subfolder represents a specific topology evaluated during the AC validation stage.
  - `{x}` is the index of the topology (e.g., `topology_0`, `topology_1`, etc.).

#### **optimizer_snapshot/run_{n}/topology_{x}/**
- **modified_network.xiidm**:  
  - The grid file representing the network after applying the topology changes for this specific solution.
- **ac_loadflow_results.csv**:  
  - Results of the AC loadflow analysis for this topology containing both the N-0 and N-1 scenarios.
- **dc_loadflow_results.csv**:  
  - Results of the DC loadflow analysis for this topology.
- **sld/{y}_sld.png**:  
  - Single line diagrams (SLDs) for substations that were split as part of the topology modification.  
  - `{y}` corresponds to the name of the split station.

#### **results/task_name/**
- This folder is reserved for storing the checkpoint results and also the tensorboard logs for each DC optimisation task, organized by task name. Note that the 'task_name' corresponds to the name of the task defined in the `dc_optimisation_config`.
---

### **Summary**

- **optimizer_snapshot**: Contains all optimization runs.
- **run_{n}**: Each run folder contains results for a specific optimizer execution.
- **res.json**: Summary of the run and references to topologies.
- **topology_{x}**: Contains all files related to a specific evaluated topology, including the modified grid, loadflow results, and diagrams.
- **results/task_name**: Contains checkpoint results and logs for each DC optimization task.

This structure ensures that all results are logically grouped and easily accessible for further analysis, visualization, or reporting.

## Visualize the results

We visualize the changes of the optimized topology by inspecting the single-line diagram of the effected substation.

In [ ]:
import pypowsybl as pp
from IPython.display import SVG, display

# Adjust run directory
run_dir = 'run_0'
# Find name of the voltage level (substation) that was adjusted
# The SLD of the adjusted substation is named as "<voltage_level_name>_sld.svg"
voltage_level_name = 'VLevel2'
sld_path = optimizer_snapshot_dir / run_dir / 'topology_0' / "sld" / f"{voltage_level_name}_sld_initial.svg"

# Load initial grid into pypowsybl
network = pp.network.load(str(iteration_name / file_path))
# Run AC load flow on initial grid to see flows in SLD
pp.loadflow.run_ac(network)

# Create SVG of initial grid's SLD for the specified substation
svg_initial = get_single_line_diagram_custom(network, 'VL2')
sld_path.parent.mkdir(parents=True, exist_ok=True)
with open(sld_path, "w", encoding="utf-8") as f:
    f.write(svg_initial._content)

# Display SLD before optimization
display(SVG(filename=sld_path))
# Display SLD after optimization
display(SVG(filename=optimizer_snapshot_dir / run_dir / 'topology_0' / "sld" / f"{voltage_level_name}_sld.svg"))


We can see that the substation `VL2` was split and the busbars have slightly different voltage levels.

We can inspect the overload energy before and after the split to see the impact on congestion.
This can be done by inspecting the `res.json` in the `run_<number>` directory.
The field `overload_energy_n_1` can be found for the `best_topos` and as a member of `initial_metrics`.

Check it out yourself to get familiar with the data that the optimizer logs and uses.

This concludes the example.
We have successfully reduced the overload energy in the grid by performing topological adjustments.